### Reproducibility
[PyTorch](https://pytorch.org/docs/stable/notes/randomness.html)

In [ ]:
import os
import random
import numpy as np
import torch

In [ ]:
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
try:
    torch.use_deterministic_algorithms(True)
except AttributeError:
    torch.set_deterministic(True)
torch.backends.cudnn.deterministic = True

### MNIST 데이터
[MNIST](http://yann.lecun.com/exdb/mnist/)  
[PyTorch](https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.MNIST)

In [ ]:
import torchvision.transforms as transforms

from torchvision.datasets import MNIST

In [ ]:
trainset = MNIST('./data', train=True, download=True, transform=transforms.ToTensor())

In [ ]:
print(type(trainset))

In [ ]:
trainset

In [ ]:
print(type(trainset.data), type(trainset.targets))

In [ ]:
print(trainset.data.size(), trainset.targets.size())

In [ ]:
print(trainset.classes)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
trainloader = DataLoader(trainset, batch_size=1024, shuffle=True)

### PyTorch 분류기

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch.nn as nn

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(28 * 28, 10)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        out = self.fc(x)
        return out

In [ ]:
model = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
EPOCHS = 30

model.train()

losses = []

for e in range(EPOCHS):

    e_loss = []

    for xs, ys in trainloader:
        xs, ys = xs.to(device), ys.to(device)
        output = model(xs)
        loss = criterion(output, ys)
        optim.zero_grad()
        loss.backward()
        optim.step()

        e_loss.append(loss.item())

    print(f'[{e+1:2d}/{EPOCHS}] CE Los {np.mean(e_loss):.4f}')

    losses.append(np.mean(e_loss))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(e_loss)
plt.show()

### 테스트

In [ ]:
testset = MNIST('./data', train=False, transform=transforms.ToTensor())

In [ ]:
testloader = DataLoader(testset, batch_size=1024)

In [ ]:
correct, total = 0, 0

model.eval()

with torch.no_grad():
    for xs, ys in testloader:
        xs, ys = xs.to(device), ys.to(device)
        output = model(xs)

        _, predicted = torch.max(output, 1)

        total += ys.size(0)
        correct += (predicted == ys).sum().item()

In [ ]:
print('Accuracy: ', 100 * correct / total)